Henry Ho 304723723

# League Rank Predictor

Uses Riot Developer's api V3 (https://developer.riotgames.com/) and seeded data to predict the "highest achieved season tier" for each player using the data from matches as features.

Features include:

Labels include:
- UNRANKED
- BRONZE
- SILVER
- GOLD
- PLATINUM
- DIAMOND
- MASTERS
- CHALLENGER

In [1]:
# Import pandas
import pandas as pd

## Create Dataframe using Riot's seeded data

In [2]:
# Creating empty dataframe
stats_df = pd.DataFrame()

### Parse seeded data

In [3]:
# Importing JSON
import json

# Function gets stats from seeded data and returns a dataframe
def get_stats(*files):
    df = pd.DataFrame()
    
    if len(files) == 1:
        df = parse_json(files)
    elif len(files) > 1:
        for file in files:
            df = df.append(parse_json(file), ignore_index=True)
            
    return df
        
# Function that parses json file into a dataframe
def parse_json(file):
    df = pd.DataFrame(columns = ['win', 'role', 'kills', 'deaths', 'assists', 'gameDuration', 'goldEarned', 'totalMinionsKilled', 'damageDealtToObjectives', 'totalDamageDealt', 'totalDamageTaken', 'wardsPlaced', 'highestAchievedSeasonTier'])
    matches = pd.read_json(path_or_buf=file)
    
    for match in matches['matches']:
        for participant in match['participants']:
            
            df = df.append({'win': participant['stats']['win'],
                            'role': participant['timeline']['role'],
                            'kills': participant['stats']['kills'],
                            'deaths': participant['stats']['deaths'],
                            'assists': participant['stats']['assists'],
                            'gameDuration': match['gameDuration'],
                            'goldEarned': participant['stats']['goldEarned'],
                            'totalMinionsKilled': participant['stats']['totalMinionsKilled'],
                            'damageDealtToObjectives': participant['stats']['damageDealtToObjectives'],
                            'totalDamageDealt': participant['stats']['totalDamageDealt'],
                            'totalDamageTaken': participant['stats']['totalDamageTaken'],
                            'wardsPlaced': participant['stats']['wardsPlaced'],
                            'highestAchievedSeasonTier': participant['highestAchievedSeasonTier']}, ignore_index=True)

    return df

In [4]:
# datasets = ['datasets/matches1.json', 'datasets/matches2.json', 'datasets/matches3.json', 
#             'datasets/matches4.json', 'datasets/matches5.json', 'datasets/matches6.json', 
#             'datasets/matches7.json', 'datasets/matches8.json', 'datasets/matches9.json', 
#             'datasets/matches10.json']

datasets = ['datasets/matches1.json', 'datasets/matches2.json', 'datasets/matches3.json',
            'datasets/matches4.json', 'datasets/matches5.json']
stats_df = get_stats(*datasets)
print(stats_df)

        win         role kills deaths assists gameDuration goldEarned  \
0      True         SOLO    17      5      25         3509      26248   
1      True    DUO_CARRY    24     14      15         3509      28906   
2      True  DUO_SUPPORT     8     17      22         3509      18845   
3      True         NONE     7      7      29         3509      19404   
4      True         SOLO    10     10      20         3509      20805   
5     False         NONE     8     15      12         3509      19921   
6     False         SOLO    14     13      12         3509      22644   
7     False  DUO_SUPPORT    11     15      16         3509      20659   
8     False    DUO_CARRY     8     12      18         3509      22493   
9     False         SOLO    12     11      13         3509      22368   
10    False         SOLO     3      8      14         3105      17936   
11    False  DUO_SUPPORT     8     10      16         3105      19876   
12    False    DUO_CARRY     9     12      14      